In [ ]:
from utils import *
from prepare_ds import *
from train_ml import *
from create_map import *

# signs = parse_tifs_from(DEFAULT_PATH["images"], "sign", force=True)
signs = parse_tifs_from(DEFAULT_PATH["images"], "sign")
signs = signs.query("type == 'sign'")

labels = parse_tifs_from(DEFAULT_PATH["labels"], "label")
labels = labels.query("type == 'label'")

In [ ]:
# Downgrade of classes in labels.
test_label = load_data_tif(DEFAULT_PATH["labels"], only_first=True)
path_labels = DEFAULT_PATH["labels"]
new_class = path_labels + "classes_matching.csv"

if test_label["array"].mean() > 6:
    downgrade_classes(src=path_labels, out=path_labels, assign_class=new_class)

In [ ]:
# TODO: RESUME to fix all pipe to learn and validate it on etalon map.
year = 2020
signs_paths = signs.query(f"year == {year} and season == '90d'")["path"].to_list()
labels_paths = labels.query(f"year == {year}")["path"].to_list()

percent = 0.05
z_x, z_y, x, y = generate_dataset(signs_paths, labels_paths, percent=percent)


In [ ]:
print(y[0]["array"].mean())
y_test = load_data_tif(labels_paths[0], only_first=True)
print(y_test["array"].mean())

In [ ]:
m = train_model(z_x, z_y)
m

In [ ]:
# Mapping by RAM tensor-images (230m)
n = f"handle_map_{percent}.tif"
create_map(x, m, n)

In [ ]:
# Mapping by loaded images from path (10m)
p = DEFAULT_PATH["images"]
n = f"handle_map_10m_{percent}.tif"
images = load_data_tif(p)

In [ ]:
create_map(images, m, n)